<a href="https://colab.research.google.com/github/lucia990/ADM_GROUP-30_HW_4/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Data import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/ADM_HW_4')

In [ ]:
!pwd

/content/drive/MyDrive/ADM_HW_4


In [ ]:
!kaggle datasets download -d shivamb/bank-customer-segmentation

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip bank-customer-segmentation.zip

Archive:  bank-customer-segmentation.zip
  inflating: bank_transactions.csv   


###Inspect the dataset

In [ ]:
#create the dataframe
import pandas as pd
data = pd.read_csv('bank_transactions.csv', usecols = ['CustomerID', 'CustomerDOB','CustGender',	'CustLocation',	'CustAccountBalance', 'TransactionDate', 'TransactionTime',	'TransactionAmount (INR)'])
#drop the columns containing transaction and customer IDs since they are not useful information for similarity detection 

In [ ]:
#group by the customer id. For each of them we pick the last transaction
customer_data = data.groupby('CustomerID').last()
customer_data.reset_index(drop = True, inplace = True)

In [ ]:
#check how many missing values there are in the dataset
customer_data.isna().sum()

CustomerDOB                2372
CustGender                  803
CustLocation                114
CustAccountBalance         1665
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

In [ ]:
customer_data.dropna(inplace= True)
customer_data.reset_index(drop = True, inplace = True)

In [ ]:
customer_data.isna().sum()

CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64

#### Transaction Date

In [ ]:
#we want the date to be of datetime type
customer_data['TransactionDate'] = pd.to_datetime(customer_data.TransactionDate)

In [ ]:
customer_data.TransactionDate

0        2016-09-08
1        2016-08-14
2        2016-07-08
3        2016-09-15
4        2016-08-18
            ...    
879354   2016-07-08
879355   2016-09-15
879356   2016-08-18
879357   2016-08-28
879358   2016-09-25
Name: TransactionDate, Length: 879359, dtype: datetime64[ns]

In [ ]:
customer_data.TransactionDate.dt.year.value_counts()

2016    879359
Name: TransactionDate, dtype: int64

We see that our dataset deals with transactions in 2016. 

#### Date of Birth

In [2]:
#we want the date to be of datetime type
customer_data['CustomerDOB'] = pd.to_datetime(customer_data.CustomerDOB)

NameError: ignored

In [ ]:
customer_data.CustomerDOB.dt.year.hist(bins= 50)

In [ ]:
customer_data.loc[customer_data.CustomerDOB.dt.year == 1800, 'CustomerDOB']

In [ ]:
customer_data.loc[customer_data.CustomerDOB.dt.year > 2016, 'CustomerDOB'].sort_values()


the DOB in 1800 have the same exact date, unlike the successive DOB, so maybe it is better to correct them with different approaches:
- for DOB > 2016  simply subtract 100 years in order to assign a likely date of birth. Of course this represents an issue for finding similar customers, but better than dropping all the information related to these transactions. 
- for DOB = **1800-01-01**  need to replace the date of birth with the customer resulting the most similar to the customers whos DOB is wrong. 

In [ ]:
customer_data.loc[customer_data.CustomerDOB.dt.year > 2016, 'CustomerDOB'] = customer_data.loc[customer_data.CustomerDOB.dt.year > 2016, 'CustomerDOB'] - pd.DateOffset(years = 100)

In [ ]:
customer_data.loc[customer_data.CustomerDOB.dt.year > 2016, 'CustomerDOB'].count()

 The chosen date of birth chosen for all these transactions will be the DOB of the customer having these common features.  

In [ ]:
#dataframe containing only the data with the wrong date of birth
#This dataframe will help us finding similar behaviors among the customers
sub_data = customer_data.loc[customer_data.CustomerDOB.dt.year == 1800]

For the majority, the wrong date are related to male customers. 

In [ ]:
sub_data.CustAccountBalance.mean()

In [ ]:
sub_data.CustLocation.value_counts()


In [ ]:
sub_data['TransactionAmount (INR)'].value_counts()

a male customer, located in New Delhi, with a transaction amount of 1000. 

In [ ]:
#create the dataframe containing the info we need
sub_data1 = customer_data[customer_data['CustAccountBalance'] >= 268667 ][customer_data['CustLocation'] == 'NEW DELHI'][customer_data['TransactionAmount (INR)'] == 1000][customer_data['CustGender'] == 'M']

In [ ]:
#we want to keep only correct data
sub_data1 = sub_data1[sub_data1['CustomerDOB'] != '1800-01-01' ]

In [ ]:
sub_data1.sort_values(by = 'CustAccountBalance')

The customer that is most similar to the mean of customers whose date of birth is wrong is the first one. We can compare, for instance, the bar plots

 replace the date **1800-01-01** with **1981-11-09**	

In [ ]:
data1 = customer_data.replace(['1800-01-01'], '1981-11-09' )

In [ ]:
data1.reset_index(drop=True, inplace = True)

In [ ]:
data1['CustomerDOB'] = pd.to_datetime(data1['CustomerDOB'])

In [ ]:
data1.CustomerDOB.hist(bins = 50)

Now the distribution of birthdays makes sense. As we expected, there is a peak in 1981. However, this shouldn't compromise the hashing procedure. 

#### Transaction Time

In [ ]:
from datetime import datetime

In [ ]:
#Transaction time is not in the format we want
data1['TransactionTime'] = data1.TransactionTime.apply(lambda row: datetime.strptime(str(row).zfill(6), '%H%M%S'))

In [ ]:
data1.TransactionTime.dt.time

0         01:12:29
1         20:44:09
2         22:03:05
3         17:02:54
4         14:11:03
            ...   
879354    00:51:22
879355    12:02:55
879356    12:25:33
879357    21:37:22
879358    22:57:32
Name: TransactionTime, Length: 879359, dtype: object

#### Transaction Date

In [ ]:
data1['TransactionDate'] = pd.to_datetime(data1.TransactionDate)

# 1.Finding Similar Costumers

## 1.1 Set up the data

In [ ]:
#the dataset we are going to use has been already prepared
data1

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,1983-05-08,M,NEW DELHI,120180.54,2016-09-08,1900-01-01 01:12:29,356.0
1,1994-07-28,M,MUMBAI,24204.49,2016-08-14,1900-01-01 20:44:09,1499.0
2,1984-08-19,M,MUMBAI,161848.76,2016-07-08,1900-01-01 22:03:05,250.0
3,1990-05-29,F,CHAMPARAN,496.18,2016-09-15,1900-01-01 17:02:54,30.0
4,1965-06-21,M,KOLKATA,87058.65,2016-08-18,1900-01-01 14:11:03,5000.0
...,...,...,...,...,...,...,...
879354,1990-12-24,M,BHIWANDI,133067.23,2016-07-08,1900-01-01 00:51:22,691.0
879355,1996-09-06,M,BANGALORE,96063.46,2016-09-15,1900-01-01 12:02:55,222.0
879356,1993-10-21,M,GUNTUR,5559.75,2016-08-18,1900-01-01 12:25:33,126.0
879357,1995-04-22,M,CHENNAI,35295.92,2016-08-28,1900-01-01 21:37:22,50.0


In [ ]:
import pickle
with open('clean_df.pkl', 'wb') as f:
  pickle.dump(data1, f)
f.close

<function BufferedWriter.close>

## 1.2 Fingerprint hashing

*1. Implement your minhash function from scratch*

#### STEP 1: Feature discretization

The features to discretize are: 
- the balance of the customer
- the period of the year (retrieved from the TransactionDate column)
- the period of the day (retrieved from the TransactionTime column)
- the amount of the transaction
- the age of the customer 


- **Customer balance**

In [1]:
from sklearn.preprocessing import KBinsDiscretizer
import pandas as pd

In [2]:
#balance account
balance = KBinsDiscretizer(n_bins = 4, encode = 'ordinal', strategy = 'quantile')

In [3]:
balance.fit(data1.CustAccountBalance.to_numpy().reshape(-1,1))

NameError: ignored

In [ ]:
data1['discrete_balance'] = balance.transform(data1.CustAccountBalance.to_numpy().reshape(-1,1))

In [ ]:
data1.discrete_balance.value_counts()


In [ ]:
print('min_class 0:', data1.loc[data1['discrete_balance'] == 0.0, 'CustAccountBalance'].min())
print('max_class 0:', data1.loc[data1['discrete_balance'] == 0.0, 'CustAccountBalance'].max())
print('min_class 1:', data1.loc[data1['discrete_balance'] == 1.0, 'CustAccountBalance'].min())
print('max_class 1:', data1.loc[data1['discrete_balance'] == 1.0, 'CustAccountBalance'].max())
print('min_class 2:', data1.loc[data1['discrete_balance'] == 2.0, 'CustAccountBalance'].min())
print('max_class 2:', data1.loc[data1['discrete_balance'] == 2.0, 'CustAccountBalance'].max())
print('min_class 3:', data1.loc[data1['discrete_balance'] == 3.0, 'CustAccountBalance'].min())
print('max_class 3:', data1.loc[data1['discrete_balance'] == 3.0, 'CustAccountBalance'].max())


 split the account balances in 4 bins 0, 1, 2 and 3, from the lowest to the greatest balances. 

The function is useful but in practice we can't use it for each feature. As a matter of fact, there is no way to give labels to bins. To avoid ambiguity among shingles we use another method. 

- **Period of the Year**

For what concern the transaction date it is not interesting to keep all the info for finding similar customers. Two customers can be related if they do transactions in the same period of the year, so we can consider for example the month of the transaction (e.g. customers buying something during Christmas are considered similar). 
Also for the transaction time the reasoning is similar: we keep only the transaction hour. 


In [ ]:
data1.columns

In [ ]:
#period of the year
labels = ['Jan-Mar', 'Apr-Jun', 'Jul-Sept', 'Oct-Dec']
data1['period_year'] = pd.cut(data1['TransactionDate'].dt.month,  4, labels = labels)

In [ ]:
data1.period_year.value_counts()

In [ ]:
data1.loc[data1['period_year']=='Oct-Dec', 'TransactionDate' ].dt.month.value_counts()

- **Period of the day**

In [ ]:
#period of the day
labels = ['Night','Morning', 'Afternoon', 'Late evening']
data1['period_day'] = pd.cut(data1['TransactionTime'].dt.hour, 4, labels = labels)

In [ ]:
data1.loc[data1['period_day']=='Night', 'TransactionTime' ].dt.hour.value_counts()

In [ ]:
data1.period_day.value_counts()

- **Amount of the transaction**

In [ ]:
data1['TransactionAmount (INR)'].describe()

In [ ]:
#amount of the transaction
labels = ['Small','Medium', 'Big']
data1['transaction_size'] = pd.qcut(data1['TransactionAmount (INR)'], q =3, labels = labels)

In [ ]:
data1.transaction_size.value_counts()

In [ ]:
#Medium transactions
data1.loc[data1['transaction_size']=='Medium', 'TransactionAmount (INR)' ].describe()

In [ ]:
#Small transactions
data1.loc[data1['transaction_size']=='Small', 'TransactionAmount (INR)' ].describe()

In [ ]:
#Big transactions
data1.loc[data1['transaction_size']=='Big', 'TransactionAmount (INR)' ].describe()

- **Customer Age**

In [ ]:
data1['cust_age'] = 2016 - data1.CustomerDOB.dt.year 

In [ ]:
data1.cust_age.value_counts()

In [ ]:
#we divide the customer age in groups 
def age_group(age):
  if age in range(0, 12):
    age_group = 'child'
  elif age in range(13, 25):
    age_group = 'teen-ager'
  elif age in range(25, 50):
    age_group = 'adult'
  else:
    age_group = 'old'
  return age_group

In [ ]:
data1['age_group'] = data1.cust_age.apply(lambda x: age_group(x))

In [ ]:
data1.age_group.value_counts()

In [ ]:
with open('custom_df.pkl', 'wb') as f: 
  pickle.dump(data1, f)
f.close

In [ ]:
import pickle
with open('custom_df.pkl', 'rb') as f:
  data1 = pickle.load(f)
f.close

In [ ]:
# Keep only discrete features
discr_df = data1[['CustGender', 'CustLocation', 'discrete_balance', 'period_year', 'period_day', 'transaction_size', 'age_group' ]]

In [ ]:
#create a pickle file containing the modified dataset
import pickle
with open('modified_df.pkl', 'wb') as f:
    pickle.dump(discr_df, f)
f.close

In [ ]:
import pickle
with open('modified_df.pkl', 'rb') as f:
  modified_df = pickle.load(f)
f.close

In [ ]:
modified_df

We have 879359 rows. Through them i have to retrieve the "shingles", i.e. the unique values a customer can have.

#### STEP 2: Find unique values

In [ ]:
all_unique_values = []
for column in modified_df:
  values = modified_df[column].unique().tolist()
  set_values = set(values)
  unique_values = list(set_values)
  all_unique_values.append(unique_values)
print(all_unique_values)

In [ ]:
unique_data = [x[i] for x in all_unique_values for i in range(len(x))]

In [ ]:
print('the number of unique unigrams is: ', len(unique_data))

In [ ]:
#we want the shingles not to be ordered
import random
random.shuffle(unique_data)

In [ ]:
with open('shingles.pkl', 'wb') as f:
    pickle.dump(unique_data, f)
f.close

#### STEP 3: Build the Characteristic matrix

In [ ]:
import pickle
import numpy as np
import pandas as pd

In [ ]:
#open the pickle file
with open('modified_df.pkl', 'rb') as f:
    modified_df = pickle.load(f)
f.close

In [ ]:
#create characteristic matrix
#we use the panda function pd.get_dummies, which convert categorical variable into dummy/indicator variables
#for each categorical variable we create a row of the characteristic matrix 
gender = pd.get_dummies(modified_df['CustGender'])
balance = pd.get_dummies(modified_df['discrete_balance'])
months = pd.get_dummies(modified_df['period_year'])
days = pd.get_dummies(modified_df['period_day'])
size = pd.get_dummies(modified_df['transaction_size'])
age = pd.get_dummies(modified_df['age_group'])


In [ ]:
char_matrix1 = np.concatenate([gender, balance, months, days, size, age], axis = 1) #horizontal_concat

In [ ]:
char_matrix1

In [ ]:
char_m = np.transpose(char_matrix1)

In [ ]:
char_m.shape

#### STEP 4: Create the Signature Matrix

Instead of picking n random permutations of rows, we pick n randomly
chosen hash functions $\ h_1, h_2, . . . , h_n$ on the rows. i construct the signature
matrix by considering each row in their given order. 

In [ ]:
import random 

In [ ]:
#define a function to generate n hash function (for simulating the effect of th epermutations of the row of the characteristic matrix)
def n_hash(row, n, p):  #p is the size of the hash table. Its value is the first prime number after the last shingle
  hash_list = []
  for _ in range(n):
    h = random.randint(1, n)*(row) % p
    hash_list.append(h)
  return hash_list

Now that i choose the hash functions we can construct the signature matrix. It will have as many rows as the number of hash functions. 

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np

In [ ]:
#pre-allocate memory for the signature matrix
signatures = np.matrix(np.ones((22, char_m.shape[1]))* np.inf)
#the signature matrix has a shape (22, 879359) and every element is infinity

In [ ]:
signatures

In [ ]:
#scan over the row of the caracteristic matrix
for row in tqdm(range(char_m .shape[0])):
  
  hash_list = n_hash(row, 750, 757)  #we choose 750 hash functions #757 is the first prime number after 500
 

  #slide over the columns of the characteristic matrix
  for col in range(char_m.shape[1]): 
    if char_m[row, col] == 1:
      #print(char_matrix[row, col])
      #scan over the rows of the signature matrix
      for i in range(signatures.shape[0]): 
        signatures[i, col] = min(hash_list[i],signatures[i, col])


In [ ]:
signatures